In [1]:
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np

# Para mostrar todas las columnas cuando se imprime un df
pd.set_option('display.max_columns', None)
plt.style.use('ggplot')

Se quiere resolver un problema de selección de características en un modelo de ML. El objetivo del problema es el de selección de características. Es decir; tenemos un problema supervisado en el que tenemos un modelo $M$,  columnas predictivas $C_1, \ldots, C_N$ y una columna a predecir $y$. El problema mulit-objetivo se plantea como el conjunto de puntos donde se busca minimizar la pérdida del modelo dado que el número de columnas usadas es el menor posible. 

Mayores n_objetivoses darán lugar a un mejor modelo (menor error) así que los objetivos suelen estar en conflicto. Si el modelo incluye una manera de seleccionar características (como regularización), entonces el error menor debería ser aquel en el que se ocupen todas las columnas.
-  Esto significa que si tu algoritmo está bien equipado para enfrentar este problema, debería de haber un miembro del conjunto de Pareto por cada número de características. 
- Tal vez así se puede caracterizar un algoritmo. Qué tanto logra resolver problemas en los que haya un punto por cada dimensión en el frente de Pareto

1. Definir un problema en la que haya características que tienen la información correcta para separar un dataset. Como 4 direcciones que están dadas por una separación normal bivariada y 6 que no están. Se debería de ir probando hasta encontrar esta configuración y el frente de Pareto sería uno por dimensión. El 7 que contiene a 4 de los separados y 3 de otros, el 6 que contiene 4 de los separados y 2 de los otros, etc.
2. Entrenar un algoritmo genético que tome las características como variable binaria de si está o no en el conjunto de entrenamiento y encuentre el conjunto y frente de Pareto. 
3. Explicar en términos de indicadores qué significa tal cosa.


In [38]:
N_cols = 10
N_separables = 4
N_no_separables = N_cols - N_separables
N_registros = 1000


X_no_separables = np.random.multivariate_normal(mean=np.zeros(N_no_separables),cov=np.identity(N_no_separables),size=N_registros)

X_separables = np.array([np.r_[np.random.normal(loc=-1,scale=0.2, size=N_registros//2),np.random.normal(loc=1,scale=0.2,size=N_registros//2)] for i in range(N_separables)]).T

X_dataset = np.c_[X_separables,X_no_separables]


In [39]:
px.scatter_matrix(X_dataset)

In [65]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


for n in range(7):
    km = KMeans(n_clusters=2,n_init='auto',random_state=420)
    n_inicio = np.random.randint(9)
    n_final = np.random.randint(low=n_inicio+1,high=10)
    print(n_inicio,n_final)
    km.fit(X_dataset[:,n_inicio:n_final])
    print(n, km.inertia_, silhouette_score(X_dataset,km.labels_))



7 9
0 1383.5171380708548 0.0705626956558213
5 9
1 3384.6194878869073 0.07209825110847597
3 5
2 1045.884855923587 0.3597058548834699
0 8
3 4241.220575535495 0.3597058548834699
7 9
4 1383.5171380708548 0.0705626956558213
5 6
5 368.9671597949174 0.06503240650141656
8 9
6 384.78452983722866 0.06423441337270357


In [90]:
from itertools import combinations

dict_list = []

for i in range(2,11):
    for comb_i in list(combinations(range(10),i)):
        km = KMeans(n_clusters=i, n_init='auto')
        X_subset = X_dataset[:,list(comb_i)]
        km.fit(X_subset)

        n_separadas = len(set(comb_i).intersection(set(range(4))))
        # print(comb_i,n_separadas)
        dict_list.append({'|Subconjunto|':i,'Subconjunto':comb_i,'N_separables':n_separadas,'Inercia':km.inertia_,'Silueta':silhouette_score(X_subset,km.labels_)})

In [91]:
pd.set_option('display.max_rows',None)
pd.DataFrame(dict_list).sort_values('Inercia',ascending=True)

,|Subconjunto|,Subconjunto,N_separables,Inercia,Silueta
2,2,"(0, 3)",2,76.830768,0.877023
0,2,"(0, 1)",2,77.036899,0.876554
10,2,"(1, 3)",2,77.638435,0.875764
1,2,"(0, 2)",2,80.094484,0.874798
17,2,"(2, 3)",2,80.696020,0.874230
9,2,"(1, 2)",2,80.902151,0.873015
46,3,"(0, 1, 3)",3,102.091458,0.546603
45,3,"(0, 1, 2)",3,105.910544,0.543030
81,3,"(1, 2, 3)",3,106.106821,0.541236
53,3,"(0, 2, 3)",3,106.528410,0.540717
